In [1]:
%load_ext autoreload
%autoreload 2

from preprocess import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical

# Second dimension of the feature is dim2
feature_dim_2 = 11

# Save data to array file first
save_data_to_array(max_len=feature_dim_2)

# # Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test()

# # Feature dimension
feature_dim_1 = 20
channel = 1
epochs = 50
batch_size = 100
verbose = 1
num_classes = 5

# Reshaping to perform 2D convolution
X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
Saving vectors of label - 'Chopin': 100%|██████████| 104/104 [02:56<00:00,  1.69s/it]
Saving vectors of label - '.ipynb_checkpoints': 0it [00:00, ?it/s]
Saving vectors of label - 'happy': 100%|██████████| 1742/1742 [00:07<00:00, 224.38it/s]


In [2]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

# Predicts one sample
def predict(filepath, model):
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    return get_labels()[0][
            np.argmax(model.predict(sample_reshaped))
    ]

# Building The Model Then Training it

In [3]:
model = get_model()
model.fit(X_train, y_train_hot, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(X_test, y_test_hot))

Train on 3295 samples, validate on 2198 samples
Epoch 1/50
3295/3295 [==============================] - 2s 701us/step - loss: 1.9565 - acc: 0.3909 - val_loss: 0.8213 - val_acc: 0.6306
Epoch 2/50
3295/3295 [==============================] - 2s 474us/step - loss: 0.8342 - acc: 0.6592 - val_loss: 0.8486 - val_acc: 0.6151
Epoch 3/50
3295/3295 [==============================] - 1s 449us/step - loss: 0.6819 - acc: 0.7439 - val_loss: 0.4061 - val_acc: 0.8681
Epoch 4/50
3295/3295 [==============================] - 2s 461us/step - loss: 0.5100 - acc: 0.8115 - val_loss: 0.4027 - val_acc: 0.8480
Epoch 5/50
3295/3295 [==============================] - 1s 450us/step - loss: 0.4229 - acc: 0.8449 - val_loss: 0.3441 - val_acc: 0.8708
Epoch 6/50
3295/3295 [==============================] - 1s 452us/step - loss: 0.3844 - acc: 0.8546 - val_loss: 0.2670 - val_acc: 0.9081
Epoch 7/50
3295/3295 [==============================] - 2s 667us/step - loss: 0.3137 - acc: 0.8841 - val_loss: 0.2908 - val_acc: 0.8949


## Prediction

In [4]:
print(predict('./test/other_test (6).mp3', model=model))

other


In [7]:
import glob
for i in glob.glob('./test/*.mp3'):
    print("{0} is {1}".format(i,predict(i, model=model)))

./test/other_test (1).mp3 is other
./test/other_test (3).mp3 is other
./test/Kevin Kern - wave.mp3 is other
./test/other_test (2).mp3 is other
./test/Kevin Kern - We Should Waltz.mp3 is .ipynb_checkpoints
./test/other_test (6).mp3 is other
./test/other_test (4).mp3 is .ipynb_checkpoints
./test/other_test (8).mp3 is .ipynb_checkpoints
./test/other_test (5).mp3 is Chopin
./test/Kevin Kern - Le Jardin.mp3 is .ipynb_checkpoints


In [6]:
for i in glob.glob('./chopin_test/*.mp3'):
    print("{0} is {1}".format(i,predict(i, model=model)))

./chopin_test/chopin_test (11).mp3 is other
./chopin_test/chopin_test (18).mp3 is other
./chopin_test/chopin_test (6).mp3 is .ipynb_checkpoints
./chopin_test/chopin_test (4).mp3 is other
./chopin_test/chopin_test (15).mp3 is .ipynb_checkpoints
./chopin_test/chopin_test (13).mp3 is other
./chopin_test/chopin_test (1).mp3 is other
./chopin_test/chopin_test (2).mp3 is other
./chopin_test/chopin_test (12).mp3 is .ipynb_checkpoints
./chopin_test/chopin_test (7).mp3 is other
./chopin_test/chopin_test (17).mp3 is other
./chopin_test/chopin_test (9).mp3 is other
./chopin_test/chopin_test (3).mp3 is other
./chopin_test/chopin_test (10).mp3 is other
./chopin_test/chopin_test (8).mp3 is .ipynb_checkpoints
./chopin_test/chopin_test (16).mp3 is other
./chopin_test/chopin_test (14).mp3 is other
./chopin_test/chopin_test (5).mp3 is other
